######Documentação
https://docs.databricks.com/pt/partners/bi/power-bi.html

###### Localizar Conexão

In [0]:
# Cluster > JDBC/ODBC 
# usar Server Hostname e HTTP Path para conexão 

###### Criando banco de Dados


In [0]:
# script da Aula (2. Criando Tabelas Via Comando SQL)
# Rodar para criar Banco de dados a partir de arquivos SCV'

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS BikesCSV;
use BikesCSV;
CREATE TABLE IF NOT EXISTS Produtos    USING csv OPTIONS ('path' '/FileStore/tables/Bikes/products.csv'    ,'header' 'true','inferSchema' 'true');
CREATE TABLE IF NOT EXISTS Clientes    USING csv OPTIONS ('path' '/FileStore/tables/Bikes/customers.csv'   ,'header' 'true','inferSchema' 'true');
CREATE TABLE IF NOT EXISTS brands      USING csv OPTIONS ('path' '/FileStore/tables/Bikes/brands.csv'      ,'header' 'true','inferSchema' 'true');
CREATE TABLE IF NOT EXISTS categories  USING csv OPTIONS ('path' '/FileStore/tables/Bikes/categories.csv'  ,'header' 'true','inferSchema' 'true');
CREATE TABLE IF NOT EXISTS order_items USING csv OPTIONS ('path' '/FileStore/tables/Bikes/order_items.csv' ,'header' 'true','inferSchema' 'true');
CREATE TABLE IF NOT EXISTS orders      USING csv OPTIONS ('path' '/FileStore/tables/Bikes/orders.csv'      ,'header' 'true','inferSchema' 'true');
CREATE TABLE IF NOT EXISTS staffs      USING csv OPTIONS ('path' '/FileStore/tables/Bikes/staffs.csv'      ,'header' 'true','inferSchema' 'true');
CREATE TABLE IF NOT EXISTS stocks      USING csv OPTIONS ('path' '/FileStore/tables/Bikes/stocks.csv'      ,'header' 'true','inferSchema' 'true');
CREATE TABLE IF NOT EXISTS stores      USING csv OPTIONS ('path' '/FileStore/tables/Bikes/stores.csv'      ,'header' 'true','inferSchema' 'true');

In [0]:
%sql
show databases


In [0]:
%sql
show tables

In [0]:
%sql 

select 
o.order_id
,i.quantity              as Quantidade
,p.product_name          as Produto
,b.brand_name            as Marca
,p.model_year            as Modelo
,ct.category_name        as Categoria
,i.list_price            as Valor
,o.order_date            as DtCompra
,c.first_name            as Cliente
,c.email                 as EmailCliente
,c.city                  as Cidade
,c.state                 as Estado
,s.store_name            as LojaCompra
,s.email                 as EmailLoja
from      order_items I
left join orders      O  on i.order_id = o.order_id
left join clientes    C  on o.customer_id = c.customer_id
left join produtos    P  on i.product_id = p.product_id
left join stores      S  on o.store_id = s.store_id
left join brands      B  on p.brand_id = b.brand_id
left join categories  CT on p.category_id = ct.category_id

######Transformando resultado da consulta em outra tabela no Banco de Dados

In [0]:
# inserir consulta em uma variavel fica mais facil
query = """

select 
o.order_id
,i.quantity              as Quantidade
,p.product_name          as Produto
,b.brand_name            as Marca
,p.model_year            as Modelo
,ct.category_name        as Categoria
,i.list_price            as Valor
,o.order_date            as DtCompra
,c.first_name            as Cliente
,c.email                 as EmailCliente
,c.city                  as Cidade
,c.state                 as Estado
,s.store_name            as LojaCompra
,s.email                 as EmailLoja
from      order_items I
left join orders      O  on i.order_id = o.order_id
left join clientes    C  on o.customer_id = c.customer_id
left join produtos    P  on i.product_id = p.product_id
left join stores      S  on o.store_id = s.store_id
left join brands      B  on p.brand_id = b.brand_id
left join categories  CT on p.category_id = ct.category_id


"""
resultado = spark.sql(query)

# Salvando em formato delta em banco de dados ("nao sabe o que é Delta? nos proximos módulos vou dar exemplos e vamos aprender na prática , não se preocupe")
resultado.write.\
    format("delta")\
    .mode("overwrite")\
    .saveAsTable("bikescsv.ConsultaCompleta")


In [0]:
%sql
select * from consultacompleta
-- ver no power bi em forma de tabela

Curiosidades para ir pesquisando 

In [0]:
#usar catalogo hive_metastore

In [0]:
"""
Vamos ver tudo isso em aulas posteriores!!!

Delta é projetado para manter e gerenciar dados de maneira transacional e eficiente. Essa pasta no DBFS contém os arquivos e metadados necessários para implementar as características do Delta, como transações ACID, controle de versão e otimizações de consulta.

A pasta Hive  é uma parte do sistema de gerenciamento de metadados do Databricks, e é usada para manter o registro das tabelas e metadados relacionados. Quando você cria uma tabela no Databricks usando o formato Delta, um registro da tabela é armazenado no Hive Metastore, que é uma parte integrante do ambiente Databricks.





"""

###### Salvando todos Arquivos no formato Delta em Banco de Dados

In [0]:
%sql
show databases

In [0]:
%sql
show tables

In [0]:
%sql
-- Criando novo banco de Dados
CREATE DATABASE IF NOT EXISTS teste;


In [0]:
%sql
--Deletar  tabelas obs(passar banco de dados primeiro.depois nome da tabela)
DROP table IF EXISTS bikescsv.brands ;

In [0]:
%sql
--Deletar todos bancos de dados
DROP DATABASE IF EXISTS teste CASCADE;


In [0]:
"""
Importanto todos os dados no formato Delta para 

Fluxo vai ser 

1° Criar novo banco de Dados
2° Transformar todos os dados de origem em um DF
3° Salvar os DataFrames no formato Delta para o banco de Dados 
Vamos Fazer na prática?

"""

In [0]:
%sql
-- 1° Criar novo banco de Dados  para receber as tabelas Delta
CREATE DATABASE IF NOT EXISTS BikesDelta;

In [0]:

#2° Transformar todos os dados de origem em um DF
produtos_df     = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/FileStore/tables/Bikes/products.csv")
clientes_df     = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/FileStore/tables/Bikes/customers.csv")
brands_df       = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/FileStore/tables/Bikes/brands.csv")
categories_df   = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/FileStore/tables/Bikes/categories.csv")
order_items_df  = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/FileStore/tables/Bikes/order_items.csv")
orders_df       = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/FileStore/tables/Bikes/orders.csv")
staffs_df       = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/FileStore/tables/Bikes/staffs.csv")
stocks_df       = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/FileStore/tables/Bikes/stocks.csv")
stores_df       = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/FileStore/tables/Bikes/stores.csv")

In [0]:
stocks_df.show(5)

In [0]:
# 3° Salvar os DataFrames no formato Delta para o banco de Dados
produtos_df.write.format("delta").mode("overwrite").saveAsTable("BikesDelta.Produtos")
clientes_df.write.format("delta").mode("overwrite").saveAsTable("BikesDelta.Clientes")
brands_df.write.format("delta").mode("overwrite").saveAsTable("BikesDelta.Brands")
categories_df.write.format("delta").mode("overwrite").saveAsTable("BikesDelta.Categories")
order_items_df.write.format("delta").mode("overwrite").saveAsTable("BikesDelta.order_items")
orders_df.write.format("delta").mode("overwrite").saveAsTable("BikesDelta.Orders")
staffs_df.write.format("delta").mode("overwrite").saveAsTable("BikesDelta.Staffs")
stocks_df.write.format("delta").mode("overwrite").saveAsTable("BikesDelta.Stocks")
stores_df.write.format("delta").mode("overwrite").saveAsTable("BikesDelta.Stores")

In [0]:
%sql
use BikesDelta;
select * from Produtos

In [0]:
%sql
select 
o.order_id
,i.quantity              as Quantidade
,p.product_name          as Produto
,b.brand_name            as Marca
,p.model_year            as Modelo
,ct.category_name        as Categoria
,i.list_price            as Valor
,o.order_date            as DtCompra
,c.first_name            as Cliente
,c.email                 as EmailCliente
,c.city                  as Cidade
,c.state                 as Estado
,s.store_name            as LojaCompra
,s.email                 as EmailLoja
from      BikesDelta.order_items I
left join BikesDelta.orders      O  on i.order_id = o.order_id
left join BikesDelta.clientes    C  on o.customer_id = c.customer_id
left join BikesDelta.produtos    P  on i.product_id = p.product_id
left join BikesDelta.stores      S  on o.store_id = s.store_id
left join BikesDelta.brands      B  on p.brand_id = b.brand_id
left join BikesDelta.categories  CT on p.category_id = ct.category_id


In [0]:
#usar catalogo hive_metastore